In [6]:
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
from glob import glob
import os
import cv2
from openslide import OpenSlide
from skimage.measure import label
import numpy as np
from matplotlib import pyplot as plt

In [2]:
img_files_path = [_ for _ in os.listdir(r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\4xr_jpg') if _.endswith(".jpg")]
img_files_path_png = [x.replace('.jpg','.png') for x in img_files_path]

In [3]:
ndpi = r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\c1'
ndpis = glob(os.path.join(ndpi,'*.ndpi'))

mask = r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\c1\1um\classification_v9_combined'
masks = glob(os.path.join(mask,'*.tif'))

binary_mask = r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\c1\TA'
binary_masks = glob(os.path.join(binary_mask,'*.png'))

### Crop whole slide image, its respective whole slide mask, and then also crop the binary mask together: (note they all have different sizes so you have to use the ratios)

In [ ]:
# for i in range(len(ndpis)):
#     wsi = OpenSlide(ndpis[i])
#     target_level = 4
#     wsi_im = wsi.read_region(location=(0,0),level=target_level,size=wsi.level_dimensions[target_level])
#
#     fn = os.path.basename(ndpis[i])
#     fn,ext = os.path.splitext(fn)
#     TAsrc = r'\\fatherserverdw\kyuex\image\CLUE\3D study\he\c1\TA'
#     TA = os.path.join(TAsrc,fn+'.png')
#     bw = Image.open(TA)
#     labeledbw = label(np.array(bw))
#
#     target_tissue = 4
#     bwtarget = labeledbw == target_tissue
#
#     ind = np.argwhere(bwtarget)
#     margin = 30
#     bbox = [np.min(ind[:,0]),np.min(ind[:,1]),np.max(ind[:,0]),np.max(ind[:,1])] #[miny,minx,maxy,maxx]
#     bboxm = [bbox[0]-margin,bbox[1]-margin,bbox[2]+margin,bbox[3]+margin]
#     bwcropm = bwtarget[bboxm[0]:bboxm[2],bboxm[1]:bboxm[3]]
#
#     wsi_arr = np.array(wsi_im)
#     wsisz = wsi_arr.shape
#     bwsz = bwtarget.shape
#     rsf = wsisz[0]/bwsz[0]
#     bboxm2 = [round(_*rsf) for _ in bboxm]
#     imcropm = wsi_arr[bboxm2[0]:bboxm2[2],bboxm2[1]:bboxm2[3]]
#
#     im_mask = np.array(Image.open(masks[i]))
#     ratio = im_mask.shape[0]/wsi_arr.shape[0]
#     immaskm = im_mask[round(bboxm2[0]*ratio):round(bboxm2[2]*ratio),round(bboxm2[1]*ratio):round(bboxm2[3]*ratio)]
#
#     im_binary_mask = np.array(Image.open(binary_masks[i]))
#     ratio = im_binary_mask.shape[0]/wsi_arr.shape[0]
#     im_binary_mask_m = im_binary_mask[round(bboxm2[0]*ratio):round(bboxm2[2]*ratio),round(bboxm2[1]*ratio):round(bboxm2[3]*ratio)]
#
#     imcropm_src = r'\\fatherserverdw\Kevin\imageregistration2\images'
#     immaskm_src = r'\\fatherserverdw\Kevin\imageregistration2\masks'
#     im_binary_mask_src = r'\\fatherserverdw\Kevin\imageregistration2\binary_mask_wsi'
#
#     Image.fromarray(imcropm).save(os.path.join(imcropm_src,img_files_path_png[i]))
#     Image.fromarray(immaskm).save(os.path.join(immaskm_src,img_files_path_png[i]))
#     Image.fromarray(im_binary_mask_m).save(os.path.join(im_binary_mask_src,img_files_path_png[i]))
#     #save imcropm,immaskm,im_binary_mask_m

### Now, Resize binary masks in binary_mask_wsi to the same size of the padded masks, which is all 5352 x 6328.

In [14]:
binary_mask_files_name = [_ for _ in os.listdir(r'\\fatherserverdw\Kevin\imageregistration2\binary_mask_wsi') if _.endswith(".png")]
bin_mask = r'\\fatherserverdw\Kevin\imageregistration2\binary_mask_wsi'
bin_masks = glob(os.path.join(bin_mask,'*.png'))
im_binary_mask_resized_src = r'\\fatherserverdw\Kevin\imageregistration2\binary_mask_wsi\resized'
padded_mask_ex = Image.open(r'\\fatherserverdw\Kevin\imageregistration2\padded_masks\z0001_1C1.png')
dim_padded_masks = padded_mask_ex.size

In [ ]:
for i in range(len(binary_mask_files_name)):
    temp = cv2.imread(bin_masks[i])
    resize_bin_mask = cv2.resize(temp,dim_padded_masks,interpolation=cv2.INTER_CUBIC)
    resize_bin_mask = cv2.cvtColor(resize_bin_mask, cv2.COLOR_BGR2GRAY)
    tmp_src = os.path.join(im_binary_mask_resized_src,binary_mask_files_name[i])
    cv2.imwrite(tmp_src,resize_bin_mask)